# Hyperparam optimisation

In [1]:
%load_ext autoreload
%autoreload 2

# %matplotlib widget

In [2]:
import pandas as pd
import warnings
import seaborn as sns

import optuna
from optuna.samplers import TPESampler
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
)

import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from xgboost import XGBRegressor
import shap

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold


# from plots2.fns import (
#     standard_layout,
#     corner_annotations_colwise,
#     corner_annotations_rowwise,
#     get_text_annotation
# )

from poly2.utils import get_dispersal_kernel, trait_vec, beta_dist, gamma_dist
from poly2.run import no_joblib_single_run, no_joblib_multiple_run
from poly2.config import Config, ConfigMixture, get_asymptote_config
from poly2.simulator import SimulatorOneTrait, SimulatorAsymptote

C:\Users\user\anaconda3\envs\poly2\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
df = pd.read_csv('../outputs/combined/scan_all.csv')

In [4]:
# df_filt = df.loc[lambda x: x.year<=30].reset_index(drop=True)

In [5]:
df.head()

,run,year,best_dose,n_pos_diff,in_0p0_0p1,in_0p1_0p2,in_0p2_0p3,in_0p3_0p4,in_0p4_0p5,in_0p5_0p6,...,in_0p8_0p9,in_0p9_1p0,mu,b,asymptote,dec_rate_multiplier,m_prop_multiplier,m_scale_multiplier,ME_var,ME_mean
0,0,1,1.0,9,1.0,5.054079e-27,7.584700e-34,1.472094e-39,4.846678e-45,9.976588e-51,...,2.492945e-74,3.501367e-90,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,5.044122e-17,0.398241
1,0,2,1.0,9,1.0,5.054079e-27,7.584700e-34,1.472094e-39,4.846678e-45,9.976588e-51,...,2.492945e-74,3.501367e-90,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,5.044122e-17,0.398241
2,0,3,1.0,9,1.0,5.054079e-27,7.584700e-34,1.472094e-39,4.846678e-45,9.976588e-51,...,2.492945e-74,3.501367e-90,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,5.044122e-17,0.398241
3,0,4,1.0,9,1.0,5.054079e-27,7.584700e-34,1.472094e-39,4.846678e-45,9.976588e-51,...,2.492945e-74,3.501367e-90,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,5.044122e-17,0.398241
4,0,5,1.0,9,1.0,5.054079e-27,7.584700e-34,1.472094e-39,4.846678e-45,9.976588e-51,...,2.492945e-74,3.501367e-90,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,5.044122e-17,0.398241


In [6]:
df.columns

Index(['run', 'year', 'best_dose', 'n_pos_diff', 'in_0p0_0p1', 'in_0p1_0p2',
       'in_0p2_0p3', 'in_0p3_0p4', 'in_0p4_0p5', 'in_0p5_0p6', 'in_0p6_0p7',
       'in_0p7_0p8', 'in_0p8_0p9', 'in_0p9_1p0', 'mu', 'b', 'asymptote',
       'dec_rate_multiplier', 'm_prop_multiplier', 'm_scale_multiplier',
       'ME_var', 'ME_mean'],
      dtype='object')

In [7]:
X = (
    # df_filt
    df
    .drop([
        'best_dose',
        'n_pos_diff',
        'ME_var',
    ], axis=1)
    .filter(regex='^((?!in_0).)*$')
    .rename(columns = {
        'dec_rate_multiplier': 'dec_rate',
        'm_prop_multiplier': 'm_prop',
        'm_scale_multiplier': 'm_scale',
        'asymptote': 'asymp',
    })
)

# y = df_filt.loc[:, ['run', 'best_dose']]
y = df.loc[:, ['run', 'best_dose']]

In [8]:
X.head()

,run,year,mu,b,asymp,dec_rate,m_prop,m_scale,ME_mean
0,0,1,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.398241
1,0,2,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.398241
2,0,3,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.398241
3,0,4,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.398241
4,0,5,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.398241


In [9]:
X.filter(like='run').describe()

,run
count,350000.000000
mean,4999.500000
std,2886.755455
min,0.000000
25%,2499.750000
50%,4999.500000
75%,7499.250000
max,9999.000000


# Fit XGBoost model and find good hyperparams

In [10]:
np.random.seed(0)

# ignore warning about Int64Index
warnings.simplefilter(action='ignore', category=FutureWarning)

Get data (by run)

In [11]:
# reset index? Might help with Int64Index XGB warning
X_cv = X.loc[lambda x: (x.run<8000)].drop('run', axis=1)
y_cv = y.loc[lambda x: (x.run<8000)].drop('run', axis=1)

X_test = X.loc[lambda x: (x.run>=8000)].drop('run', axis=1)
y_test = np.array(y.loc[lambda x: (x.run>=8000)].drop('run', axis=1))

How do the default params do?

In [ ]:
%%time

rmse_list = []
        
kf = KFold(n_splits=10)

for train_ind, val_ind in kf.split(X_cv):

    X_tr = X_cv.iloc[train_ind]
    y_tr = y_cv.iloc[train_ind]

    X_v = X_cv.iloc[val_ind]
    y_v = y_cv.iloc[val_ind]
    
    train_runs = X.iloc[train_ind].run
    val_runs = X.iloc[val_ind].run

    print(f'ok? {sum(train_runs.isin(val_runs))==0}')

    y_tr = np.array(y_tr)
    y_v = np.array(y_v)

    model = XGBRegressor().fit(X_tr, y_tr)

    y_p = model.predict(X_v)

    rmse = mean_squared_error(y_p, y_v, squared=False)

    rmse_list.append(rmse)

score = sum(rmse_list)/len(rmse_list)

score

ok? True


Including `in_0p1...` we got `0.12505` on the default params. Without does `0.1301`.

## Hyperparam opt

In [ ]:
class HyperparamsObj:
    def __init__(self, X_in, y_in) -> None:
        self.X = X_in
        self.y = y_in
        
    #
    #

    def __call__(self, trial):

        params = self.get_params(trial)

        score = self.run_model(params)

        return score

    #
    #

    def run_model(self, params):
        
        X, y = self.X, self.y
        
        rmse_list = []
        
        kf = KFold(n_splits=5)

        for train_ind, val_ind in kf.split(X):
            
            X_tr = X.iloc[train_ind]
            y_tr = y.iloc[train_ind]
            
            X_v = X.iloc[val_ind]
            y_v = y.iloc[val_ind]
            
            y_tr = np.array(y_tr)
            y_v = np.array(y_v)
        
            model = XGBRegressor(**params).fit(X_tr, y_tr)

            y_p = model.predict(X_v)
            
            rmse = mean_squared_error(y_p, y_v, squared=False)

            rmse_list.append(rmse)
        
        score = sum(rmse_list)/len(rmse_list)

        return score

    #
    #

    def get_params(self, trial):
        params = {
            "tree_method": "hist",
            "max_depth": trial.suggest_int("max_depth", 3, 20),
            "n_estimators": trial.suggest_int("n_estimators", 10, 1000, log=True),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1),
        }
        return params

In [ ]:
optuna.logging.set_verbosity(0)

In [ ]:
obj = HyperparamsObj(X_cv, y_cv)

sampler = TPESampler(seed=10)
study = optuna.create_study(sampler=sampler)

NB takes about 17 hours

In [ ]:
%%time
study.optimize(obj, n_trials=10)
study.best_value

In [ ]:
%%time
study.optimize(obj, n_trials=90)
study.best_value

In [ ]:
%%time
study.optimize(obj, n_trials=100)
study.best_value

In [ ]:
%%time
study.optimize(obj, n_trials=100)
study.best_value

Previous model (with weird pars) got best value of `0.1042` after 91 trials out of 200 trials.

## Plots

In [ ]:
f = plot_optimization_history(study)
f.update_layout(height=500)

In [ ]:
f = plot_contour(study)
f.update_layout(height=800)

## Check performance of best model

Check model doesn't drastically degrade on completely unseen data

Prev version got `0.1019` on train set and `0.1261` on the test set.

In [ ]:
best_pars = study.best_params

best_pars

In [ ]:
best_model = XGBRegressor(**best_pars).fit(X_cv, y_cv)

y_p = best_model.predict(X_test)

rmse = mean_squared_error(y_p, y_test, squared=False)

rmse

In [ ]:
# model = XGBRegressor().fit(X_train_val, y_train_val)
model = XGBRegressor().fit(X_cv, y_cv)

y_p = model.predict(X_test)

rmse = mean_squared_error(y_p, y_test, squared=False)

rmse

### Save model

In [ ]:
best_model.save_model('xgb_scan_all.json')